# Classifier

### Import dataset

In [6]:
import pandas as pd

file_path = "../data/catering_reviews.csv"  # Replace with your actual file path
data = pd.read_csv(file_path)

if data is not None:
    print("Processed Data:")
    print(data.head())
else:
    print("No data to process.")

Processed Data:
     author  rating                                               text  \
0   Estelle     5.0  Monsieur M a surpassé toutes nos attentes lors...   
1  Patricia     5.0  Nous recommandons vivement les services de Mon...   
2     Julie     4.8  Un grand merci à Camille et à toute l’équipe d...   
3  Amandine     5.0  L’équipe était très professionnel, souriante e...   
4  Mathilde     5.0  Accompagnement en amont, prestation le jour J,...   

         date sentiment        source                company_id company_name  
0  2024-04-11  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
1  2025-05-25  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
2  2024-12-10  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
3  2024-11-09  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
4  2025-02-02  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
